<a href="https://colab.research.google.com/github/Anichan55/Twitter-Stock-Sentiment-Analysis/blob/main/Twitter_Stock_Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install flair

     |████████████████████████████████| 319 kB 5.8 MB/s 
     |████████████████████████████████| 52 kB 1.0 MB/s 
     |████████████████████████████████| 981 kB 43.3 MB/s 
     |████████████████████████████████| 2.9 MB 26.4 MB/s 
     |████████████████████████████████| 788 kB 77.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 19.7 MB 1.5 MB/s 
     |████████████████████████████████| 64 kB 4.3 MB/s 
     |████████████████████████████████| 48 kB 8.0 MB/s 
     |████████████████████████████████| 1.2 MB 53.5 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 636 kB 36.6 MB/s 
     |████████████████████████████████| 3.3 MB 43.4 MB/s 
     |████████████████████████████████| 895 kB 24.8 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9704 sha256=4bf16229d948fc99e2a009ec2ff8e86be7bcb7f63e3b1d

# **Importing** **Libraries**

In [3]:

import numpy as np
import matplotlib.pyplot as plt

import string
import random
import re
import requests
import os
import textwrap

from datetime import datetime, timedelta
import pandas as pd

# **Defining** **functions**

In [13]:
bearer_token='AAAAAAAAAAAAAAAAAAAAAIRUSQEAAAAAWLbfpT20bMYhL2hRtEq3aEXHefU%3DjI5tBIHPozlQZZ4WNskkSU9BBawkyOlrNT6hr9KCdspbhoui0n'
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

    params = {
    'tweet_mode': 'extended',
    'lang': 'en',
    'count': '100'
}



def get_data(tweet):
    data = {
        'id': tweet['id_str'],
        'created_at': tweet['created_at'],
        'text': tweet['full_text']
    }
    return data

dtformat = '%Y-%m-%dT%H:%M:%SZ'  # the date format string required by twitter

# we use this function to subtract 60 mins from our datetime string
def time_travel(now, mins):
    now = datetime.strptime(now, dtformat)
    back_in_time = now - timedelta(minutes=mins)
    return back_in_time.strftime(dtformat)
    
now = datetime.now()  # get the current datetime, this is our starting point
last_week = now - timedelta(days=2)  # datetime 2 days ago= the finish line
now = now.strftime(dtformat)  # convert now datetime to format for API

df = pd.DataFrame()  # initialize dataframe to store tweets
while True:
    if datetime.strptime(now, dtformat) < last_week:
        # if we have reached 2 days ago, break the loop
        break
    pre60 = time_travel(now, 60)  # get 60 minutes before 'now'
    # assign from and to datetime parameters for the API
    response = requests.get('https://api.twitter.com/1.1/search/tweets.json?q=birla', auth=bearer_oauth,params = {'tweet_mode': 'extended','lang': 'en','count': '100','start_time':pre60,'end_time':now})  # send the request
    now = pre60  # move the window 60 minutes earlier
    # iteratively append our tweet data to our dataframe
    for tweet in response.json()['statuses']:
      row = get_data(tweet)
      df = df.append(row, ignore_index=True)

In [14]:
tweets=df
tweets['sentiment'] = np.nan
tweets['probability']=np.nan
len(tweets)

16800

# **Initialising Model using Flair**

In [6]:
import flair
sentiment_model = flair.models.TextClassifier.load('en-sentiment')


2021-10-04 01:00:19,265 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpt7ib8dlc


100%|██████████| 265512723/265512723 [00:16<00:00, 16558032.52B/s]

2021-10-04 01:00:35,798 copying /tmp/tmpt7ib8dlc to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2021-10-04 01:00:36,560 removing temp file /tmp/tmpt7ib8dlc
2021-10-04 01:00:37,445 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [15]:
i=0
for t in tweets['text']:
  whitespace = re.compile(r"\s+")
  web_address = re.compile(r"(?i)http(s):\/\/[a-z0-9.~_\-\/]+")
  user = re.compile(r"(?i)@[a-z0-9_]+")
  RT=re.compile(r"RT :")
# we then use the sub method to replace anything matching
  tweet = whitespace.sub(' ', t)
  tweet = web_address.sub('', tweet)
  tweet = user.sub('', tweet)
  tweet = RT.sub('',tweet)
  tweets['text'][i]=tweet
  i=i+1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [16]:
len(tweets)

16800

In [17]:
for i in range(0,len(tweets)):
  sentence = flair.data.Sentence(tweets['text'][i])
  sentiment_model.predict(sentence)
  probability = sentence.labels[0].score  # numerical value 0-1
  sentiment = sentence.labels[0].value  # 'POSITIVE' or 'NEGATIVE'
  tweets['sentiment'][i]=sentiment
  tweets['probability'][i]=probability
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [18]:
tweets

,created_at,id,text,sentiment,probability
0,Mon Oct 04 01:18:17 +0000 2021,1444834222410280961,Karnataka HC bars Aditya Birla Fashion to sel...,NEGATIVE,0.998931
1,Mon Oct 04 00:44:37 +0000 2021,1444825752021598209,Goenka Birla sunil mittal all are cry…,POSITIVE,0.976250
2,Sun Oct 03 23:59:05 +0000 2021,1444814293304856580,Please,POSITIVE,0.563061
3,Sun Oct 03 23:54:43 +0000 2021,1444813192505483271,"After having fully considered the question, I...",NEGATIVE,0.947012
4,Sun Oct 03 21:42:52 +0000 2021,1444780012838612992,Recently Paid Tributes at the compound of Bir...,POSITIVE,0.977484
...,...,...,...,...,...
16795,Sun Oct 03 13:34:56 +0000 2021,1444657222273802244,"Don't listen to what they say,Just go &am...",NEGATIVE,0.688065
16796,Sun Oct 03 13:17:46 +0000 2021,1444652899829628933,Delhi | Lok Sabha Speaker Om Birla pays tribu...,POSITIVE,0.953341
16797,Sun Oct 03 13:09:15 +0000 2021,1444650757932797962,Don’t need to give her weightage.H...,NEGATIVE,0.999705
16798,Sun Oct 03 12:53:15 +0000 2021,1444646729773568010,#Telangana #Taliban #films #politics #Dollywoo...,NEGATIVE,0.999960


# **Calculating the overall sentiment of the stock over the last week**

In [19]:
avg_positive_sentiment=sum(tweets['probability'][tweets['sentiment']=='POSITIVE'])/sum(tweets['sentiment']=='POSITIVE')
avg_negative_sentiment=sum(tweets['probability'][tweets['sentiment']=='NEGATIVE'])/sum(tweets['sentiment']=='NEGATIVE')
total_positive_sentiment=sum(tweets['probability'][tweets['sentiment']=='POSITIVE'])
total_negative_sentiment=sum(tweets['probability'][tweets['sentiment']=='NEGATIVE'])

In [20]:
(total_positive_sentiment - total_negative_sentiment)

5770.461354732513